In [1]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [2]:
data = pd.read_csv("news.csv")

In [3]:
print(data.shape)
data.head()

(6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
labels = data.label
x_train,x_test,y_train,y_test=train_test_split(data['text'], labels, test_size=0.2, random_state=5)

# TF-IDF and Variable Selection (Pre-Processing)

In [5]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

#Variable Selection
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile = 10)
selector.fit(tfidf_train, y_train)
tfidf_train_trans = selector.transform(tfidf_train).toarray()
tfidf_test_trans = selector.transform(tfidf_test).toarray()

# Passive Agressive Classifier

In [6]:
#PAC with variable selection
#DataFlair - Initialize a PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train_trans,y_train)

#DataFlair - Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test_trans)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.37%


In [7]:
#PAC without variable selection
#DataFlair - Initialize a PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)

#DataFlair - Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 94.16%


In [8]:
#DataFlair - Build confusion matrix
print(confusion_matrix(y_test,y_pred, labels=['FAKE','REAL']))

[[599  38]
 [ 36 594]]


# Linear Discriminant Analysis

In [9]:
lda = LinearDiscriminantAnalysis()
lda.fit(tfidf_train_trans, y_train.ravel())
lda_pred = lda.predict(tfidf_test_trans)
lda_acc = accuracy_score(y_test, lda_pred)
print(lda_acc)

NameError: name 'LinearDiscriminantAnalysis' is not defined

In [ ]:
lda_confusion_matrix = confusion_matrix(y_test, lda_pred)
lda_confusion_matrix

# Quadratic Discriminant Analysis

In [ ]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(tfidf_train_trans, y_train.ravel())
qda_pred = qda.predict(tfidf_test_trans)
qda_acc = accuracy_score(y_test, qda_pred)
print(qda_acc)

In [ ]:
qda_confusion_matrix = confusion_matrix(y_test, qda_pred)
qda_confusion_matrix

# Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(tfidf_train_trans, y_train)
rf_pred = rf.predict(tfidf_test_trans)
rf_acc = accuracy_score(y_test, rf_pred)
print(rf_acc)

In [ ]:
rf_confusion_matrix = confusion_matrix(y_test, rf_pred)
rf_confusion_matrix

# Support Vector Machine

In [ ]:
svm = SVC()
svm.fit(tfidf_train_trans, y_train)
svm_pred = svm.predict(tfidf_test_trans)
svm_acc = accuracy_score(y_test, svm_pred)
print(svm_acc)

In [ ]:
svm_confusion_matrix = confusion_matrix(y_test, svm_pred)
svm_confusion_matrix

# Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(tfidf_train_trans, y_train)
nb_pred = nb.predict(tfidf_test_trans)
nb_acc = accuracy_score(y_test, nb_pred)
print(nb_acc)

In [ ]:
svm_confusion_matrix = confusion_matrix(y_test, svm_pred)
svm_confusion_matrix

# Model Comparison (Cross-Validation)

In [ ]:
import matplotlib
from sklearn.model_selection import cross_val_score

In [ ]:
model_names = {'PAC', 'QDA', 'RF', 'SVM', 'NB'}
models = pd.DataFrame([[0,0,0,0,0]],columns = model_names)
toappend = models
for i in range(9):
    models = models.append(toappend, ignore_index = True)
print(models)

In [ ]:
data = pd.read_csv("news.csv")

In [ ]:
text = data['text'] 
labels = data.label

tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
#DataFlair - Fit and transform train set, transform test set
tfidf_x = tfidf_vectorizer.fit_transform(text) 

In [ ]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile = 10)
selector.fit(tfidf_x, labels)
tfidf_x_trans = selector.transform(tfidf_x).toarray()

In [ ]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac_cv = cross_val_score(pac, tfidf_x_trans, labels, cv=10)

In [ ]:
qda = QuadraticDiscriminantAnalysis()
qda_cv = cross_val_score(qda, tfidf_x_trans, labels, cv=10)

In [ ]:
rf = RandomForestClassifier()
rf_cv = cross_val_score(rf, tfidf_x_trans, labels, cv=10)

In [ ]:
svm = SVC()
svm_cv = cross_val_score(svm, tfidf_x_trans, labels, cv=10)

In [ ]:
nb = GaussianNB()
nb_cv = cross_val_score(nb, tfidf_x_trans, labels, cv=10)

In [ ]:
models['PAC'] = pac_cv
models['QDA'] = qda_cv
models['RF'] = rf_cv
models['SVM'] = svm_cv
models['NB'] = nb_cv
print(models)

In [ ]:
models.to_csv('model_evaluation_accuracy_table.csv', index = False)

In [ ]:
import matplotlib
boxplot = models.boxplot()